<h1 style="text-align: center;">Data Understanding - CASH-OUT

## Importando Libraries e Dados

In [1]:
## Importando Pacotes
import time
tic = time.time()

import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import duckdb
from tqdm import tqdm
from matplotlib.ticker import FormatStrFormatter    
import re
from validate_docbr import CPF, CNPJ
from sklearn.model_selection import train_test_split

from utils.feature_enginneering import *

## Configs
warnings.filterwarnings('ignore')
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)

seed = 123

In [2]:
## Importando Dataset Geral
file_path1 = 'dados/df_cashout_limpo.parquet'
df1 = pd.read_parquet(f'{file_path1}')

file_path2 = 'dados/df_cashout_limpo2.parquet'
df2 = pd.read_parquet(f'{file_path2}')

columns_not_in_df1 = df2.columns.difference(df1.columns)

In [3]:
## criação da
df2['last_status'] = np.where(df2['last_treatment_status'].isin(['CONFIRMED_FRAUD']), 'FRAUD',
                            np.where(df2['last_treatment_status'].isin(['SUSPECTED_FRAUD']), 'SUSPECTED_FRAUD',
                            df2['last_status']))

df2.drop(columns_not_in_df1, axis='columns', inplace=True)
list(df1.columns) == list(df2.columns)

df = pd.concat([df1, df2], ignore_index=True)
df['last_status'].value_counts(dropna=False)

# df = df2.copy()

last_status
APPROVED           10101858
REJECTED              17541
SUSPECTED_FRAUD         255
FRAUD                    35
Name: count, dtype: int64

In [4]:
df.sort_values('last_status_changed_at', ascending=False, inplace=True)
df.drop_duplicates(subset='transaction_id', keep='first', inplace=True)
df.duplicated('transaction_id').sum()

0

In [5]:
df[['transaction_amount_total', 'transaction_amount_total_brl']].describe()

,transaction_amount_total,transaction_amount_total_brl
count,"10,110,814.00","5,516,835.00"
mean,893.78,907.74
std,"9,530.74","10,272.51"
min,0.01,0.01
25%,23.54,12.66
50%,110.00,100.96
75%,370.70,350.00
max,"5,000,000.00","5,000,000.00"


In [6]:
df['transaction_amount_total'].sum()

9036893441.569994

In [7]:
## Importando Dataset de Fraudes
file_path_fraud  = 'dados/cashout_fraudes_id.csv'
df_fraud = pd.read_csv(file_path_fraud)
# df_fraud['fraud'] = 1

## check coluna de ID da transação
df_fraud['End2end da transação'].duplicated().sum()


def limpa_campo_moeda(value):
    # Remove the currency symbol
    value = value.replace("R$", "").strip()
    # Replace the period with an empty string to remove thousands separator
    value = value.replace(".", "")
    # Replace the comma with a dot for decimal conversion
    value = value.replace(",", ".")
    return float(value)

# Apply the function to the 'Valor da transação' column
df_fraud['Valor da transação'] = df_fraud['Valor da transação'].apply(limpa_campo_moeda)


In [8]:
colunas_fraud = ['End2end da transação', 'payouts - End2end da transação → payout_id', 'payouts - End2end da transação → pix_payout_id']
colunas_delorean = ['transaction_id','internal_id',  ]

for coluna_fraud in colunas_fraud:
    for coluna_delorean in colunas_delorean:

        df_fraud[coluna_fraud] = [str(i).strip() for i in df_fraud[coluna_fraud]]
        df[coluna_delorean] = [str(i).strip() for i in df[coluna_delorean]]
        print(f'{coluna_fraud} + {coluna_delorean}')
        print(f'rows df_fraud: {(df_fraud[df_fraud[coluna_fraud].isin(df[coluna_delorean])]).shape[0]}, rows df_delorean: {(df[df[coluna_delorean].isin(df_fraud[coluna_fraud])]).shape[0]}')
        print('-'*80)

End2end da transação + transaction_id
rows df_fraud: 16, rows df_delorean: 16
--------------------------------------------------------------------------------
End2end da transação + internal_id
rows df_fraud: 0, rows df_delorean: 0
--------------------------------------------------------------------------------
payouts - End2end da transação → payout_id + transaction_id
rows df_fraud: 35, rows df_delorean: 35
--------------------------------------------------------------------------------
payouts - End2end da transação → payout_id + internal_id
rows df_fraud: 0, rows df_delorean: 0
--------------------------------------------------------------------------------
payouts - End2end da transação → pix_payout_id + transaction_id
rows df_fraud: 0, rows df_delorean: 0
--------------------------------------------------------------------------------
payouts - End2end da transação → pix_payout_id + internal_id
rows df_fraud: 0, rows df_delorean: 0
------------------------------------------------

In [9]:
df_teste = df[df['transaction_id'].isin(df_fraud['payouts - End2end da transação → payout_id'])]
df_teste.sort_values('transaction_amount_total', ascending=False).head(5)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl
1502172,2024-06-30 22:23:00.990266,42277fa1-d0d5-5958-80f4-c9777d1317f3,transfeera,1ef372f4-f6d5-652c-af1e-9cbd84767de1,,BRL,"25,000.00",2024-06-30 22:22:25.934,cash-out,pix,848509d1-67be-4b90-8e04-5be48cd48240,TAON DELIVERY,4331422532,33292501000148,86015400,Londrina,Av Carlos Gomes,38,Frente,BR,None,593,[],[],None,...,None,[CNPJ],[55503083000123],NaT,2024-06-30 22:23:00.990419,RULE,2024-07-02 18:55:44.077556,MANUAL,FRAUD,2024-07-02 18:55:44.077556,MANUAL,"[754, 758, 755]",Marcos Machioni,marcos.machioni@transfeera.com,None,1,cnpj,33292501000148,None,None,None,None,None,None,NaN
1502186,2024-06-30 22:23:38.479807,a91f8200-b373-556e-a2d2-ae0e4a7fa009,transfeera,1ef372f6-5906-6aba-9a2c-f8249079d1bc,,BRL,"25,000.00",2024-06-30 22:23:31.641,cash-out,pix,848509d1-67be-4b90-8e04-5be48cd48240,TAON DELIVERY,4331422532,33292501000148,86015400,Londrina,Av Carlos Gomes,38,Frente,BR,None,593,[],[],None,...,None,[CPF],[58708580898],NaT,2024-06-30 22:23:38.480025,RULE,2024-07-02 18:55:51.244618,MANUAL,FRAUD,2024-07-02 18:55:51.244618,MANUAL,"[754, 755, 758]",Marcos Machioni,marcos.machioni@transfeera.com,None,1,cnpj,33292501000148,None,None,None,None,None,None,NaN
1502187,2024-06-30 22:27:02.146216,89afb288-5f2b-59cc-8309-f68e8f8e0304,transfeera,1ef372fd-f215-662d-af1e-82a63a686d5e,,BRL,"25,000.00",2024-06-30 22:26:22.406,cash-out,pix,848509d1-67be-4b90-8e04-5be48cd48240,TAON DELIVERY,4331422532,33292501000148,86015400,Londrina,Av Carlos Gomes,38,Frente,BR,None,593,[],[],None,...,None,[CPF],[58708580898],NaT,2024-06-30 22:27:02.146374,RULE,2024-07-02 18:55:59.196714,MANUAL,FRAUD,2024-07-02 18:55:59.196714,MANUAL,"[758, 755, 754]",Marcos Machioni,marcos.machioni@transfeera.com,None,1,cnpj,33292501000148,None,None,None,None,None,None,NaN
1502201,2024-06-30 22:27:46.774139,13c9e266-c002-5b5a-a0c6-ca759e08285e,transfeera,1ef372ff-9c1c-6b84-9a2c-74616d01fa73,,BRL,"25,000.00",2024-06-30 22:27:39.957,cash-out,pix,848509d1-67be-4b90-8e04-5be48cd48240,TAON DELIVERY,4331422532,33292501000148,86015400,Londrina,Av Carlos Gomes,38,Frente,BR,None,593,[],[],None,...,None,[CNPJ],[55503083000123],NaT,2024-06-30 22:27:46.774284,RULE,2024-07-02 18:56:06.492842,MANUAL,FRAUD,2024-07-02 18:56:06.492842,MANUAL,"[754, 758, 755]",Marcos Machioni,marcos.machioni@transfeera.com,None,1,cnpj,33292501000148,None,None,None,None,None,None,NaN
1502238,2024-06-30 22:30:25.891798,7587e138-46b1-56e8-b915-88cf48fada3b,transfeera,1ef37305-898b-6bf1-af1e-584adb3e11f9,,BRL,"25,000.00",2024-06-30 22:30:18.443,cash-out,pix,848509d1-67be-4b90-8e04-5be48cd48240,TAON DELI

In [10]:
df_teste2= df_fraud[df_fraud['payouts - End2end da transação → payout_id'].isin(df['transaction_id'])]
df_teste2.sort_values('Valor da transação', ascending=False).head(5)

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id
45,solicitado,TAON DELIVERY TECNOLOGIA LTDA,33292501000148,"1 Jul, 2024, 11:51",E270840982024063022237PJzPCd083n,"25,000.00","30 Jun, 2024, 19:23",55503083 JEFFERSON CRUZ REIS JUNIOR,55503083000123,CONTA_PAGAMENTO,58983739,1,290.00,8561701,Pagseguro Internet S.A,1ef372f4-f6d5-652c-af1e-9cbd84767de1,1ef372f4-fd6b-681a-b9dd-c269d6517d61
44,solicitado,TAON DELIVERY TECNOLOGIA LTDA,33292501000148,"1 Jul, 2024, 11:51",E27084098202406302223WD8qdBdlCrq,"25,000.00","30 Jun, 2024, 19:23",klaiver santos de jesus,58708580898,CONTA_PAGAMENTO,52527509,1,290.00,8561701,Pagseguro Internet S.A,1ef372f6-5906-6aba-9a2c-f8249079d1bc,1ef372f6-61aa-68a6-96d5-ebc9b8a20ad3
43,solicitado,TAON DELIVERY TECNOLOGIA LTDA,33292501000148,"1 Jul, 2024, 11:51",E27084098202406302227MOUiOq3vmG8,"25,000.00","30 Jun, 2024, 19:27",klaiver santos de jesus,58708580898,CONTA_PAGAMENTO,52527509,1,290.00,8561701,Pagseguro Internet S.A,1ef372fd-f215-662d-af1e-82a63a686d5e,1ef372fd-f81a-67e6-b9dd-7948f0c8d39a
42,solicitado,TAON DELIVERY TECNOLOGIA LTDA,33292501000148,"1 Jul, 2024, 11:51",E27084098202406302227aI5GejV686X,"25,000.00","30 Jun, 2024, 19:27",55503083 JEFFERSON CRUZ REIS JUNIOR,55503083000123,CONTA_PAGAMENTO,58983739,1,290.00,8561701,Pagseguro Internet S.A,1ef372ff-9c1c-6b84-9a2c-74616d01fa73,1ef372ff-a1cc-6347-b9dd-ab8797f4f7ed
41,solicitado,TAON DELIVERY TECNOLOGIA LTDA,33292501000148,"1 Jul, 2024, 11:51",E27084098202406302230K2NaLhGqIK5,"25,000.00","30 Jun, 2024, 19:30",BRUNA CATALAO MACHADO GUIMARAES,58913037874,CONTA_PAGAMENTO,12188722,1,NaN,24313102,Bepay instituicao de pagamento s.a.,1ef37305-898b-6bf1-af1e-584adb3e11f9,1ef37305-8f65-66eb-b9dd-e8560772e286



Antes do dia 16 de julho enviamos o payout_id  
Do dia 16 de julho até o dia 24 de julho enviamos pix_payout_id  
Depois do dia 24 somente end2end_id  

In [11]:
## End2end da transação 
## payouts - End2end da transação → payout_id

df_deltatime = pd.concat([df_teste[['transaction_id', 'payer_cnpj', 'transaction_datetime_request', 'transaction_amount_total', 'last_status']].sort_values('transaction_amount_total', ascending=False).reset_index(drop=True), 
           df_teste2[['payouts - End2end da transação → payout_id', 'CNPJ da conta', 'Transação finalizada em', 'Valor da transação']].sort_values('Valor da transação', ascending=False).reset_index(drop=True)], axis='columns').sort_values(['last_status', 'transaction_amount_total'], ascending=False)

df_deltatime['Transação finalizada em'] = pd.to_datetime(df_deltatime['Transação finalizada em'])

df_deltatime['time_delta'] =  df_deltatime['transaction_datetime_request'] - df_deltatime['Transação finalizada em']
df_deltatime

,transaction_id,payer_cnpj,transaction_datetime_request,transaction_amount_total,last_status,payouts - End2end da transação → payout_id,CNPJ da conta,Transação finalizada em,Valor da transação,time_delta
0,1ef372f4-f6d5-652c-af1e-9cbd84767de1,33292501000148,2024-06-30 22:22:25.934,"25,000.00",FRAUD,1ef372f4-f6d5-652c-af1e-9cbd84767de1,33292501000148,2024-06-30 19:23:00,"25,000.00",0 days 02:59:25.934000
1,1ef372f6-5906-6aba-9a2c-f8249079d1bc,33292501000148,2024-06-30 22:23:31.641,"25,000.00",FRAUD,1ef372f6-5906-6aba-9a2c-f8249079d1bc,33292501000148,2024-06-30 19:23:00,"25,000.00",0 days 03:00:31.641000
2,1ef372fd-f215-662d-af1e-82a63a686d5e,33292501000148,2024-06-30 22:26:22.406,"25,000.00",FRAUD,1ef372fd-f215-662d-af1e-82a63a686d5e,33292501000148,2024-06-30 19:27:00,"25,000.00",0 days 02:59:22.406000
3,1ef372ff-9c1c-6b84-9a2c-74616d01fa73,33292501000148,2024-06-30 22:27:39.957,"25,000.00",FRAUD,1ef372ff-9c1c-6b84-9a2c-74616d01fa73,33292501000148,2024-06-30 19:27:00,"25,000.00",0 days 03:00:39.957000
4,1ef37305-898b-6bf1-af1e-584adb3e11f9,33292501000148,2024-06-30 22:30:18.443,"25,000.00",FRAUD,1ef37305-898b-6bf1-af1e-584adb3e11f9,33292501000148,2024-06-30 19:30:00,"25,000.00",0 days 03:00:18.443000
...,...,...,...,...,...,...,...,...,...,...
18,1ef42bbd-1c1d-665b-bd84-7c6ec74e4c25,46250852000138,2024-07-15 15:06:30.890,583.37,APPROVED,1ef42bbd-1c1d-665b-bd84-7c6ec74e4c25,46250852000138,2024-07-15 12:06:00,583.37,0 days 03:00:30.890000
22,1ef42bbe-972a-695d-b6e7-540f942a4571,46250852000138,2024-07-15 15:07:10.892,326.04,APPROVED,1ef42bbe-972a-695d-b6e7-540f942a4571,46250852000138,2024-07-15 12:07:00,326.04,0 days 03:00:10.892000
23,1ef42bc0-d264-6e13-b6e7-2e6a24f89cac,46250852000138,2024-07-15 15:08:10.508,326.01,APPROVED,1ef42bc0-d264-6e13-b6e7-2e6a24f89cac,46250852000138,2024-07-15 12:08:00,326.01,0 days 03:00:10.508000
25,1ef42bbd-da7e-6ec9-b6e7-5416ee782d19,46250852000138,2024-07-15 15:06:50.853,292.91,APPROVED,1ef42bbd-da7e-6ec9-b6e7-5416ee782d19,46250852000138,2024-07-15 12:06:00,292.91,0 days 03:00:50.853000


In [12]:
pd.crosstab(df['last_status'], df['last_status_changed_by'])

last_status_changed_by,ALGORITHM,MANUAL,RULE,THIRD_PARTY
last_status,,,,
APPROVED,5558072,1,4534910,0
FRAUD,5,25,4,1
REJECTED,0,0,17541,0
SUSPECTED_FRAUD,0,0,255,0


In [13]:
pd.crosstab(df['last_status'], df['last_status_changed_by'], normalize='index')*100

last_status_changed_by,ALGORITHM,MANUAL,RULE,THIRD_PARTY
last_status,,,,
APPROVED,55.07,0.00,44.93,0.00
FRAUD,14.29,71.43,11.43,2.86
REJECTED,0.00,0.00,100.00,0.00
SUSPECTED_FRAUD,0.00,0.00,100.00,0.00


### ESTUDOS Delta entre TRX

In [14]:
df['transaction_datetime_request'] = pd.to_datetime(df['transaction_datetime_request'])
df['transaction_datetime_request'].describe()

count                         10110814
mean     2024-08-25 09:58:48.006620672
min                2024-01-01 03:00:59
25%      2024-07-19 22:22:02.249999872
50%         2024-08-26 19:02:50.500000
75%      2024-10-02 03:07:51.880999936
max         2024-11-06 22:53:21.075000
Name: transaction_datetime_request, dtype: object

In [15]:
df = df.sort_values(by=['transaction_payer_registration_data_doc', 
                        'transaction_receiver_registration_data_doc', 
                        'transaction_datetime_request'])
df['time_diff'] = df.groupby(
    ['transaction_payer_registration_data_doc', 'transaction_receiver_registration_data_doc']
)['transaction_datetime_request'].diff()
tempo_medio_entre_trx = df.groupby(
    ['transaction_payer_registration_data_doc', 'transaction_receiver_registration_data_doc']
)['time_diff'].mean()
pd.DataFrame(tempo_medio_entre_trx).describe(percentiles=[.1,.25,.5, .75, .9, .99])

,time_diff
count,1034690
mean,15 days 13:18:17.437699342
std,17 days 07:47:01.076114339
min,0 days 00:00:00
10%,0 days 00:00:39.471700
25%,2 days 11:48:44.773500
50%,10 days 02:16:33.752500
75%,23 days 19:21:21.800308333
90%,35 days 02:58:44.872199999
99%,81 days 00:10:05.934319999


In [16]:

df = df.sort_values(by=['transaction_payer_registration_data_doc', 
                        'transaction_datetime_request'])
df['time_diff'] = df.groupby(
    ['transaction_payer_registration_data_doc']
)['transaction_datetime_request'].diff()
tempo_medio_entre_trx_all = df.groupby(
    ['transaction_payer_registration_data_doc']
)['time_diff'].mean()
pd.DataFrame(tempo_medio_entre_trx_all).describe(percentiles=[.1,.25,.5, .75, .9, .99])

,time_diff
count,859
mean,3 days 17:41:44.088534550
std,8 days 13:58:40.006452538
min,0 days 00:00:00
10%,0 days 00:08:51.646678860
25%,0 days 00:40:35.333685686
50%,0 days 03:21:25.236734917
75%,1 days 10:33:26.889068867
90%,15 days 15:19:09.007844444
99%,32 days 01:41:34.200920


In [17]:
df_fraud['Transação finalizada em'].min(), df_fraud['Transação finalizada em'].max()


('10 Feb, 2024, 10:10', '9 Feb, 2024, 16:41')

In [18]:
df['approved_at'].dropna().sort_values()

86         2024-05-28 15:01:25.170153
522        2024-05-31 12:52:58.779746
442        2024-05-31 12:53:42.106726
444        2024-05-31 12:53:45.178511
446        2024-05-31 12:53:46.324017
                      ...            
10118507   2024-11-06 22:53:24.771201
10118512   2024-11-06 22:53:25.330272
10118508   2024-11-06 22:53:25.382134
10118528   2024-11-06 22:53:25.416005
10118529   2024-11-06 22:53:25.429051
Name: approved_at, Length: 10093012, dtype: datetime64[ns]

In [19]:
df['hora'] = pd.to_datetime(df['transaction_datetime_request']).dt.hour.astype(int)
df['dia'] = pd.to_datetime(df['transaction_datetime_request']).dt.day.astype(int)
df['dia_semana'] = pd.to_datetime(df['transaction_datetime_request']).dt.dayofweek

In [20]:
df[df['transaction_amount_total_brl'] <= 1].head(5)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl,time_diff,hora,dia,dia_semana
4603516,2024-08-20 10:49:46.268433,a3ced799-3792-5b1f-a20d-4919116c87f7,transfeera,E27084098202408201049vJmrdMn5uOX,,BRL,0.01,2024-08-20 10:49:45.287,cash-out,pix,d5201e8c-f20f-489a-9609-1879e4140e47,ASSOCIACAO DE POUPANCA E EMPRESTIMO POUPEX,,00655522000121,70630902,BRASILIA,AV DUQUE DE CAXIAS,0,parte A,BR,CHECKING_ACCOUNT,593,[],[],50533088968,...,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-20 10:49:46.268664,ALGORITHM,[],None,None,None,1,cnpj,00655522000121,cpf,50533088968,{},None,None,null,0.01,0 days 14:05:49.228000,10,20,1
4607655,2024-08-20 12:15:27.563870,8356bf2b-8070-526f-a647-16ea65aa2134,transfeera,E27084098202408201215mE4ywfFUUER,,BRL,0.01,2024-08-20 12:15:26.139,cash-out,pix,d5201e8c-f20f-489a-9609-1879e4140e47,ASSOCIACAO DE POUPANCA E EMPRESTIMO POUPEX,,00655522000121,70630902,BRASILIA,AV DUQUE DE CAXIAS,0,parte A,BR,CHECKING_ACCOUNT,593,[],[],00212562177,...,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-20 12:15:27.564189,ALGORITHM,[],None,None,None,1,cnpj,00655522000121,cpf,00212562177,{},None,None,null,0.01,0 days 01:25:40.852000,12,20,1
4607970,2024-08-20 12:21:25.582324,4852b097-53cc-5c6d-8671-5e88506398ac,transfeera,E27084098202408201221EKCpo5QDops,,BRL,0.01,2024-08-20 12:21:24.510,cash-out,pix,d5201e8c-f20f-489a-9609-1879e4140e47,ASSOCIACAO DE POUPANCA E EMPRESTIMO POUPEX,,00655522000121,70630902,BRASILIA,AV DUQUE DE CAXIAS,0,parte A,BR,CHECKING_ACCOUNT,593,[],[],27341151153,...,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-20 12:21:25.582570,ALGORITHM,[],None,None,None,1,cnpj,00655522000121,cpf,27341151153,{},None,None,null,0.01,0 days 00:05:58.371000,12,20,1
4608049,2024-08-20 12:23:07.541690,3c12204c-afc5-5024-923b-a7d500905871,transfeera,E27084098202408201223CJIrJfnU0qo,,BRL,0.01,2024-08-20 12:23:06.371,cash-out,pix,d5201e8c-f20f-489a-9609-1879e4140e47,ASSOCIACAO DE POUPANCA E EMPRESTIMO POUPEX,,00655522000121,70630902,BRASILIA,AV DUQUE DE CAXIAS,0,parte A,BR,CHECKING_ACCOUNT,593,[],[],42090903104,...,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-20 12:23:07.541912,ALGORITHM,[],None,None,None,1,cnpj,00655522000121,cpf,42090903104,{},None,None,null,0.01,0 days 00:01:41.861000,12,20,1
4608111,2024-08-20 12:24:31.904610,2a3a92a6-08be-53e1-9297-92010ef2c183,transfeera,E270840982024082012245AK9MWhuCtr,,BRL,0.01,2024-08-20 12:24:30.411,cash-out,pix,d5201e8c-f20f-489a-9609-1879e4140e47,ASSOCIACAO DE POUPANCA E EMPRESTIMO POUPEX,,00655522000121,70630902,BRASILIA,AV DUQUE DE CAXIAS,0,parte A,BR,CHECKING_ACCOUNT,593,[],[],54320135172,...,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-20 12:24:31.904871,ALGORITHM,[],None,None,None,1,cnpj,00655522000121,cpf,54320135172,{},None,None,null,0.01,0 days 00:01:24.040000,12

In [21]:
df['test_transaction'] = np.where(df['transaction_amount_total'] <= 1, 1, 0)
df['test_transaction'].sum()

1428850

In [22]:
# Ordenando valores do dataframe por transaction_receiver_registration_data_doc e transaction_datetime_request
df = df.sort_values(by=['payer_cnpj', 'transaction_datetime_request'])

In [23]:
df['last_status'].value_counts()

last_status
APPROVED           10092983
REJECTED              17541
SUSPECTED_FRAUD         255
FRAUD                    35
Name: count, dtype: int64

In [24]:
# def calcula_percentil(group):
#     return group.quantile(0.99)

# # Separa obaseado na coluna boleana
# stats = {}
# for value in [1, 0]:
#     stats[value] = df_feat_engineering[df_feat_engineering['fraud'] == value].drop(columns='fraud').agg(calcula_percentil)

# # converte para DF 
# df_mediana = pd.DataFrame(stats).T

# df_mediana.loc[:, df_mediana.columns.str.startswith('sum')]

## Analise Manual - Fraudes

In [25]:
fraudes_trx = df[df['last_status'] == 'FRAUD']
fraudes_trx['transaction_id'] = [i.lower() for i in fraudes_trx['transaction_id'] ]
fraudes_trx['transaction_id'] = [re.sub(r'-', '', i) for i in fraudes_trx['transaction_id'] ]
fraudes_trx['transaction_id'] = [ re.sub(r'\d', '', i, count=1)for i in fraudes_trx['transaction_id'] ]
# fraudes_trx.head(2)

df_fraud['End2end da transação'] = [i.lower() for i in df_fraud['End2end da transação'] ]
df_fraud['End2end da transação'] = [re.sub(r'-', '', i) for i in df_fraud['End2end da transação'] ]
df_fraud['End2end da transação'] = [ re.sub(r'\d', '', i, count=1)for i in df_fraud['End2end da transação'] ]

df_fraud['CNPJ da conta'] = [str(i) for i in df_fraud['CNPJ da conta']]
# df_fraud.head(2)

In [26]:
df_fraud[df_fraud['CNPJ da conta'] == '46250852000138']

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id
16,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151509nzl7nlzpqq7,"3,329.43","15 Jul, 2024, 12:09",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bc2-ae31-60b3-b6e7-2d32c36b72cb,1ef42bc2-c182-6d48-b4d7-8509827f7e15
17,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151508c1eulcg6ojc,326.01,"15 Jul, 2024, 12:08",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bc0-d264-6e13-b6e7-2e6a24f89cac,1ef42bc0-d960-6417-b4d7-126bc3a47d7c
18,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151507hr7st5qjloy,326.04,"15 Jul, 2024, 12:07",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbe-972a-695d-b6e7-540f942a4571,1ef42bbe-9e6f-680f-8b45-b30b84025a88
19,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e70840982024071515071i3lhmnhmbt,112.26,"15 Jul, 2024, 12:07",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbe-35d3-6a45-bd84-601d726dc913,1ef42bbe-3bd5-68a7-b4d7-7e1ab85e203f
20,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e70840982024071515061h8nm8vqqe3,292.91,"15 Jul, 2024, 12:06",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbd-da7e-6ec9-b6e7-5416ee782d19,1ef42bbd-e093-625e-b4d7-2b22acb492a7
21,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151506ibin3earqzg,583.37,"15 Jul, 2024, 12:06",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbd-1c1d-665b-bd84-7c6ec74e4c25,1ef42bbd-27cb-6546-8b45-95f82503360e
22,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151459qbqqf2beqbf,"1,689.37","15 Jul, 2024, 11:59",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bad-d4ca-6e35-bd84-1622d8c374bd,1ef42bad-dcaa-65f8-8b45-429558cbc09f
23,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151450qgmcdfhzyyq,"3,298.86","15 Jul, 2024, 11:50",MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42b98-39ba-6c5b-b6e7-665fcbe7d826,1ef42b98-4075-6997-b4d7-8cd7579c6bc3
24,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e70840982024071513502vocamazh1o,"3,102.93","15 Jul, 2024, 10:50",Matheus souza dos santos,11700719459,CONTA_PAGAMENTO,3272965,500,NaN,17192451,Banco itaucard s.a.,1ef42b12-233f-626e-bd84-c9769cbe0969,1ef42b12-28f4-6cac-8b45-fa7c65a68d58
25,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",e7084098202407151350of6uds31ntw,"6,285.31","15 Jul, 2024, 10:50",Matheus souza dos santos,11700719459,CONTA_PAGAMENTO,3272965,500,NaN,17192451,Banco itaucard s.a.,1ef42b12-233f-6272-bd84-12bba74d69be,1ef42b12-2e34-6203-b4d7-3fd206eab6e2


In [27]:
king_cash = df[df['transaction_payer_registration_data_doc'] == '46250852000138']
print(king_cash[king_cash['transaction_amount_total'].isin(df_fraud['Valor da transação'])]['last_status'].value_counts())

king_cash['transaction_id'] = [i.lower() for i in king_cash['transaction_id'] ]
king_cash['transaction_id'] = [re.sub(r'-', '', i) for i in king_cash['transaction_id'] ]
king_cash['transaction_id'] = [ re.sub(r'\d', '', i, count=1)for i in king_cash['transaction_id'] ]

# king_cash[(king_cash['transaction_amount_total'].isin(df_fraud['Valor da transação'])) & (king_cash['transaction_description'] != '')]
# df_fraud[df_fraud['End2end da transação'].isin(king_cash['transaction_id'])]
df['transaction_datetime_request'].min(), df['transaction_datetime_request'].max()

last_status
APPROVED    45
Name: count, dtype: int64


(Timestamp('2024-01-01 03:00:59'), Timestamp('2024-11-06 22:53:21.075000'))

## Join Tabelas

In [28]:
df_fraud['Transação finalizada em'] = pd.to_datetime(df_fraud['Transação finalizada em'], format='%d %b, %Y, %H:%M')
df_fraud['Transação finalizada em'] = df_fraud['Transação finalizada em'].dt.strftime('%Y-%m-%d')
df_fraud['Transação finalizada em'] = pd.to_datetime(df_fraud['Transação finalizada em'])

df_fraud['Transação finalizada em'].head()

0   2024-10-25
1   2024-10-23
2   2024-10-23
3   2024-10-23
4   2024-10-23
Name: Transação finalizada em, dtype: datetime64[ns]

In [29]:
df['transaction_datetime_request'] = df['transaction_datetime_request'].dt.strftime('%Y-%m-%d')
df['transaction_datetime_request'].head()

60381     2024-06-05
95333     2024-06-07
95892     2024-06-07
111005    2024-06-07
95929     2024-06-07
Name: transaction_datetime_request, dtype: object

In [30]:
teste_df = df[['transaction_datetime_request', 'transaction_payer_registration_data_doc', 'transaction_amount_total']]
teste_df.shape

(10110814, 3)

In [31]:
teste_df['chave'] = teste_df['transaction_payer_registration_data_doc'].astype(str) + "_" + teste_df['transaction_amount_total'].astype(str) + "_" + teste_df['transaction_datetime_request'].astype(str)
df_fraud['chave'] = df_fraud['CNPJ da conta'].astype(str) + "_" + df_fraud['Valor da transação'].astype(str) + "_" + df_fraud['Transação finalizada em'].astype(str)


In [32]:
df['chave'] = df['transaction_payer_registration_data_doc'].astype(str) + "_" + df['transaction_amount_total'].astype(str) + "_" + df['transaction_datetime_request'].astype(str)

In [33]:
teste_df['chave'] = [i.lstrip('0') for i in teste_df['chave']]
teste_df['chave'] 

60381       169900000167_120.0_2024-06-05
95333       169900000167_120.0_2024-06-07
95892       169900000167_100.0_2024-06-07
111005      169900000167_120.0_2024-06-07
95929        169900000167_18.0_2024-06-07
                       ...               
17624      44075336000199_20.0_2024-06-03
17623      44075336000199_20.0_2024-06-03
17637     44075336000199_120.0_2024-06-03
17625     44367636000141_84.76_2024-06-03
17626     46201083000188_176.0_2024-06-03
Name: chave, Length: 10110814, dtype: object

In [34]:
# teste_df[teste_df['transaction_payer_registration_data_doc'] == ]

In [35]:
df_fraud['chave'].head()

0     4721736000100_3000.0_2024-10-25
1    45810578000141_2310.0_2024-10-23
2    45810578000141_9200.0_2024-10-23
3    45810578000141_6000.0_2024-10-23
4    45810578000141_2000.0_2024-10-23
Name: chave, dtype: object

In [36]:
# df_fraud[df_fraud['CNPJ da conta'].isin(teste_df['transaction_payer_registration_data_doc'])]
# df_fraud[df_fraud['Transação finalizada em'].isin(teste_df['transaction_datetime_request'])]
# df_fraud[df_fraud['Valor da transação'].isin(teste_df['transaction_amount_total'])]

In [37]:
teste_join = pd.merge(teste_df, df_fraud, on=['chave']  , how='left')
teste_join.shape

(10110854, 21)

In [38]:
teste_join.drop_duplicates().shape

(4212290, 21)

In [39]:
6348698 - 2786684

3562014

### DF Transações 

In [40]:
df['transaction_amount_total_brl'].describe(percentiles=[.25, .75, .90, .999])

count   5,516,835.00
mean          907.74
std        10,272.51
min             0.01
25%            12.66
50%           100.96
75%           350.00
90%         1,300.00
99.9%      72,101.00
max     5,000,000.00
Name: transaction_amount_total_brl, dtype: float64

In [41]:
value_counts_by_bins(df, 'transaction_amount_total_brl', n_bins=10)

range
0.00-500000.00           5516640
500000.00-1000000.00         136
1000000.00-1500000.00         20
1500000.00-2000000.00         21
2000000.00-2500000.00          7
2500000.00-3000000.00          1
3000000.00-3500000.00          3
3500000.00-4000000.00          2
4000000.00-4500000.00          2
4500000.00-5000000.00          3
Name: count, dtype: int64

In [42]:
# plotar_distribuicao_transacao(df, ['transaction_amount_total_brl'], .99)

#### Analise Velocidade

In [43]:
df_analise_velocidade = df[['transaction_id', 'transaction_amount_total', 'transaction_datetime_request', 'transaction_payer_registration_data_doc',  'transaction_payer_registration_data_name', 'transaction_receiver_registration_data_doc', 'transaction_receiver_registration_data_name']]
df_analise_velocidade

,transaction_id,transaction_amount_total,transaction_datetime_request,transaction_payer_registration_data_doc,transaction_payer_registration_data_name,transaction_receiver_registration_data_doc,transaction_receiver_registration_data_name
60381,1ef237af-cdab-6119-b081-6e056192b1f6,120.00,2024-06-05,00169900000167,PRIMICIA SERVICOS E PROMOCOES DE EVENTOS LTDA,49859497877,Brenda Beatriz Bezerra Da Silva
95333,1ef24d20-b647-6d64-8d01-e8ad97ad7078,120.00,2024-06-07,00169900000167,PRIMICIA SERVICOS E PROMOCOES DE EVENTOS LTDA,49859497877,Brenda Beatriz Bezerra Da Silva
95892,1ef24d20-b5b6-6a6c-a368-8fdb029e0c30,100.00,2024-06-07,00169900000167,PRIMICIA SERVICOS E PROMOCOES DE EVENTOS LTDA,40128251867,RENE PEREIRA DA SILVA
111005,1ef250cf-9b4e-6e69-8d01-77083239b934,120.00,2024-06-07,00169900000167,PRIMICIA SERVICOS E PROMOCOES DE EVENTOS LTDA,42114278816,JULIO CESAR RATTO MARTINS
95929,1ef24d24-e3c2-6e29-a368-66ae2b542a00,18.00,2024-06-07,00169900000167,PRIMICIA SERVICOS E PROMOCOES DE EVENTOS LTDA,18169766877,Denis da Silva
...,...,...,...,...,...,...,...
17624,1ef21e57-70bf-6d5e-ace4-d1885f771c34,20.00,2024-06-03,44075336000199,WIM SERVICOS LTDA,44075336000199,WIM SERVICOS LTDA
17623,1ef21e57-70c4-6c30-ace4-94d80bfa02fe,20.00,2024-06-03,44075336000199,WIM SERVICOS LTDA,44075336000199,WIM SERVICOS LTDA
17637,1ef21e57-7130-6982-b1bd-14fb16f39a83,120.00,2024-06-03,44075336000199,WIM SERVICOS LTDA,44075336000199,WIM SERVICOS LTDA
17625,1ef21e57-8a71-60f8-b1bd-8fd2d4360b68,84.76,2024-06-03,44367636000141,AURA PAY,54767259000191,JDL NEGOCIOS DIGITAIS LTDA


In [44]:
sql = duckdb.connect()
df_feat_engineering = sql.query(
"""
SELECT *, 
    COUNT(transaction_id) OVER (
        PARTITION BY transaction_payer_registration_data_doc, transaction_receiver_registration_data_doc
        ORDER BY transaction_datetime_request
    ) AS count,   

    SUM(transaction_amount_total) OVER (
        PARTITION BY  transaction_payer_registration_data_doc, transaction_receiver_registration_data_doc
        ORDER BY transaction_datetime_request
        ) AS sum,   
        
    AVG(transaction_amount_total) OVER (
        PARTITION BY  transaction_payer_registration_data_doc, transaction_receiver_registration_data_doc
        ORDER BY transaction_datetime_request
        ) AS avg,   
        
    MEDIAN(transaction_amount_total) OVER (
        PARTITION BY  transaction_payer_registration_data_doc, transaction_receiver_registration_data_doc
        ORDER BY transaction_datetime_request
        ) AS median,   



FROM df_analise_velocidade
ORDER BY transaction_payer_registration_data_doc, transaction_datetime_request;
"""
).df().dropna(axis='rows')

df_feat_engineering = df_feat_engineering[ df_feat_engineering['transaction_payer_registration_data_doc'] !=  df_feat_engineering['transaction_receiver_registration_data_doc'] ]

In [45]:
df_feat_engineering.sort_values('count', ascending=False)

,transaction_id,transaction_amount_total,transaction_datetime_request,transaction_payer_registration_data_doc,transaction_payer_registration_data_name,transaction_receiver_registration_data_doc,transaction_receiver_registration_data_name,count,sum,avg,median
1590642,E27084098202411060118RwFJgx09zOy,86.80,2024-11-06,17991841000100,koin,10573521000191,PIX Marketplace,13244,"5,239,092.28",395.58,256.74
1590527,E27084098202411061606jOxysXHQWtd,825.54,2024-11-06,17991841000100,koin,10573521000191,MERCADOPAGO.COM REPRESENTACOES LTDA,13244,"5,239,092.28",395.58,256.74
1590497,E270840982024110613566x9M4JQ9wnH,99.98,2024-11-06,17991841000100,koin,10573521000191,PIX Marketplace,13244,"5,239,092.28",395.58,256.74
1590496,E27084098202411061353rvqNbfbMyBo,"1,322.78",2024-11-06,17991841000100,koin,10573521000191,PIX Marketplace,13244,"5,239,092.28",395.58,256.74
1590495,E27084098202411061230mDu15mpUZNO,459.68,2024-11-06,17991841000100,koin,10573521000191,MERCADOPAGO.COM REPRESENTACOES LTDA.,13244,"5,239,092.28",395.58,256.74
...,...,...,...,...,...,...,...,...,...,...,...
4486674,1ef3aa7c-fc9c-6694-80e2-79e6cf47a0a1,200.00,2024-07-05,31703183000135,Facio Pagamentos LTDA,04779956595,Camila Bastos de Souza,1,200.00,200.00,200.00
4486673,1ef3b29d-45f4-6974-80e2-7b5ce5904233,200.00,2024-07-05,31703183000135,Facio Pagamentos LTDA,43974438823,Matheus Oliveira Santos,1,200.00,200.00,200.00
7773450,E27084098202409062138mXWpSoJmJpV,70.60,2024-09-06,43012440000171,MPCB,62791036920,Valdemir Fausto Ferreira,1,70.60,70.60,70.60
4486672,1ef3ae8e-7193-6152-b312-e93a1393c7d5,280.00,2024-07-05,31703183000135,Facio Pagamentos LTDA,74144588149,Walas Rodrigues de aguiar,1,280.00,280.00,280.00


In [46]:
agregacoes = df_feat_engineering.groupby(['transaction_payer_registration_data_doc', 'transaction_receiver_registration_data_doc']).agg({
    'count': ['count'],
    # 'sum': ['sum'],   
    'avg': ['mean'],
    'median': ['median'],
})

agregacoes = agregacoes[agregacoes.index.get_level_values('transaction_receiver_registration_data_doc') != '']
agregacoes

count  \
                                                                                   count   
transaction_payer_registration_data_doc transaction_receiver_registration_data_doc         
00169900000167                          01188990829                                   13   
                                        03427019837                                    2   
                                        03513517874                                   13   
                                        03705020609                                    1   
                                        03859265881                                   18   
...                                                                                  ...   
86703337000180                          99672790015                                    2   
                                        99703840078                                    4   
                                        99894610900                                    1   
                                        99903300000                                    2   
                                        99991314091                                    2   

                                                                                      avg  \
                                                                                     mean   
transaction_payer_registration_data_doc transaction_receiver_registration_data_doc          
00169900000167                          01188990829                                 73.74   
                                        03427019837                                 89.88   
                                        03513517874                                 90.08   
                                        03705020609                                200.00   
                                        03859265881                                100.22   
...                                                                                   ...   
86703337000180                          99672790015                                260.48   
                                        99703840078                                160.23   
                                        99894610900                                392.48   
                                        99903300000                                401.28   
                                        99991314091                                412.58   

                                                                                   median  
                                                                                   median  
transaction_payer_registration_data_doc transaction_receiver_registration_data_doc         
00169900000167                          01188990829                                 90.50  
                                        03427019837                                 89.88  
                                        03513517874                                 90.50  
                                        03705020609                                200.00  
                                        03859265881                                 91.50  
...                                                                                   ...  
86703337000180                          99672790015                                260.48  
                                        99703840078                                160.23  
                                        99894610900                                392.48  
                                        99903300000                                401.28  
                                        99991314091                                412.58  

[2335749 rows x 3 columns]

## DF FRAUDE

In [47]:
# df_fraud['Valor da transação'].describe(percentiles=[.25, .50, .75, .90, .99])

In [48]:
# df_fraud['Valor da transação'].value_counts_by_bins().sort_index(ascending=False)
# value_counts_by_bins(df_fraud, 'Valor da transação', n_bins=10)

In [49]:
# plotar_distribuicao_transacao(df_fraud, ['Valor da transação'], .7)

In [50]:
df[df['last_status'].isin(['SUSPECTED_FRAUD'])].sort_values(['transaction_payer_registration_data_doc', 'transaction_amount_total'], ascending=False).head(5)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl,time_diff,hora,dia,dia_semana,test_transaction,chave,range
7924199,2024-10-04 15:07:37.942173,67a4a022-5455-5f23-89e4-1405dee83bb8,transfeera,1ef82626-4860-67bf-849a-53fca02ec892,,BRL,"1,000,000.00",2024-10-04,cash-out,pix,1ef66d28-61f7-63d8-944d-0e949e87d0d1,COINS.XYZ DIGITAL MARKETS LTDA,,46919983000165,29101435,Vila Velha,Rua Inácio Higino,0,SALA 503 TORRE LESTE,BR,CHECKING_ACCOUNT,593,[],[],None,...,None,SUSPECTED_FRAUD,2024-10-04 15:07:37.942430,RULE,[755],None,None,None,1,cnpj,46919983000165,None,None,"{""operation_type"": ""transfer"", ""source"": ""APP""}",10a1ae2c-23c2-56d0-93e4-9d47cee537a0,None,null,"1,000,000.00",0 days 00:00:00,15,4,4,0,46919983000165_1000000.0_2024-10-04,500000.00-1000000.00
7924210,2024-10-04 15:07:37.988564,de4356b5-cbe4-5c6d-8ba7-18a760151d69,transfeera,1ef82626-48d4-6d06-ac1e-6299a92a39a4,,BRL,"639,320.00",2024-10-04,cash-out,pix,1ef66d28-61f7-63d8-944d-0e949e87d0d1,COINS.XYZ DIGITAL MARKETS LTDA,,46919983000165,29101435,Vila Velha,Rua Inácio Higino,0,SALA 503 TORRE LESTE,BR,CHECKING_ACCOUNT,593,[],[],None,...,None,SUSPECTED_FRAUD,2024-10-04 15:07:37.988788,RULE,[755],None,None,None,1,cnpj,46919983000165,None,None,"{""operation_type"": ""transfer"", ""source"": ""APP""}",10a1ae2c-23c2-56d0-93e4-9d47cee537a0,None,null,"639,320.00",0 days 00:00:00,15,4,4,0,46919983000165_639320.0_2024-10-04,500000.00-1000000.00
7208581,2024-09-30 10:00:04.559477,0d7a9811-e1ea-54bc-8ec7-304312b9d57b,transfeera,E27084098202409301000rO4EzqHtSyh,Pagamento para solicitação de id 66f785a1f36ce...,BRL,"24,421.74",2024-09-30,cash-out,pix,1ef66cc0-3714-6973-944d-543423fcd7ef,INNER TECNOLOGIA LTDA,34992955152,44395434000103,01405100,são paulo,rua pamplona,191,apto 33,BR,CHECKING_ACCOUNT,593,[],[],18567507790,...,None,SUSPECTED_FRAUD,2024-09-30 10:00:04.559940,RULE,[754],None,None,LOW,1,cnpj,44395434000103,cpf,18567507790,"{""operation_type"": ""transfer"", ""source"": ""API""}",0162a8e7-9fea-55e5-86f4-3beeb8e91ceb,ad692128-cfae-5fcb-990e-6353c74186f7,null,"24,421.74",0 days 00:05:46.099000,0,30,0,0,44395434000103_24421.74_2024-09-30,0.00-500000.00
7915516,2024-10-04 14:29:56.422073,c295eeeb-6243-5320-8c7a-258551f65623,transfeera,1ef825d2-085f-6e2b-ac1e-b39fce7ac55a,,BRL,"125,057.50",2024-10-04,cash-out,pix,a047dfa0-7294-4bd3-b8bc-4a240cf21f0a,INOVAH SOLUCOES & MARKETING LTDA,1159901390,44097674000121,05005030,SAO PAULO,R PALESTRA ITALIA,87,CONJ 52,BR,CHECKING_ACCOUNT,593,[],[],None,...,None,SUSPECTED_FRAUD,2024-10-04 14:29:56.422853,RULE,[755],None,None,None,1,cnpj,44097674000121,None,None,"{""operation_type"": ""transfer"", ""source"": ""API""}",02f610a8-856e-5ba7-bb74-4444f2399aae,None,null,"125,057.50",0 days 00:00:00,14,4,4,0,44097674000121_125057.5_2024-10-04,0.0

In [51]:
df_fraud.sort_values(['CNPJ da conta', 'Valor da transação'], ascending=False)

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id,chave
54,solicitado,Aiqfome LTDA,9186786000158,"26 Mar, 2024, 16:17",e7084098202403260318txa55zdjnbg,150.00,2024-03-26,Vera Lucia Rodrigues Zonzini,97649520897,CONTA_PAGAMENTO,18310862,1,301.00,13370835,BPP Instituição de Pagamento S.A.,nan,nan,9186786000158_150.0_2024-03-26
34,solicitado,Royalt Tech LTDA,51957764000100,"3 Jul, 2024, 17:56",e7084098202407031532prfmc6g4mw3,"2,224.27",2024-07-03,PAULO TSUCHIYA,6571971909,CONTA_CORRENTE,1078778,4573,33.00,90400888,Santander,1ef39516-9471-60d2-aea7-40f3ed67e67f,1ef39516-a061-66fd-ade7-c23d3a635581,51957764000100_2224.27_2024-07-03
46,solicitado,Royalt Tech LTDA,51957764000100,"18 Jun, 2024, 14:32",e708409820240617195887srlh1odh5,"2,000.00",2024-06-17,Lucas Hariel dos santos,55053937800,CONTA_PAGAMENTO,53857563,1,290.00,8561701,Pagseguro Internet S.A,1ef2ce3e-cc19-6941-af1e-6d42889f8c15,1ef2ce3e-d179-69de-96d5-893a023b53ad,51957764000100_2000.0_2024-06-17
32,solicitado,Royalt Tech LTDA,51957764000100,"3 Jul, 2024, 17:56",e7084098202407031534hkc2led4ymc,821.30,2024-07-03,PAULO TSUCHIYA,6571971909,CONTA_CORRENTE,1078778,4573,33.00,90400888,Santander,1ef3951c-5e96-6f47-98ad-3858ab979701,1ef3951c-64fb-61b7-902b-c5f99ab27897,51957764000100_821.3_2024-07-03
33,solicitado,Royalt Tech LTDA,51957764000100,"3 Jul, 2024, 17:56",e7084098202407031533jjlphl4xyex,464.40,2024-07-03,PAULO TSUCHIYA,6571971909,CONTA_CORRENTE,1078778,4573,33.00,90400888,Santander,1ef3951a-a0a2-6a71-98ad-175ca850016e,1ef3951a-afbf-6e71-ade7-281059a7c11d,51957764000100_464.4_2024-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,solicitado,BRUNO S. IKEJIRI DIFERENTE EVENTOS,17718506000133,"16 Nov, 2023, 14:41",e7084098202311142131lsnnpzbx0tj,220.00,2023-11-14,JHONATAN PEREIRA BARCELOS,70260107271,CONTA_PAGAMENTO,17339086,1,290.00,8561701,Pagseguro Internet S.A,nan,nan,17718506000133_220.0_2023-11-14
120,solicitado,Perez & Filho LTDA,17450002000185,"16 Nov, 2023, 15:56",e7084098202311161828gvcxikjtxun,"22,043.00",2023-11-16,Caroliny Firmino de Oliveira,48815566813,CONTA_PAGAMENTO,91604782,1,260.00,18236120,Nu Pagamentos S.A.,nan,nan,17450002000185_22043.0_2023-11-16
122,solicitado,Perez & Filho LTDA,17450002000185,"16 Nov, 2023, 15:56",e7084098202311161828xtbzhaf3sm5,"13,673.90",2023-11-16,Giovana Aparecida Moreira de Siqueira,51687384860,CONTA_POUPANCA,64258083,1,260.00,18236120,Nu Pagamentos S.A.,nan,nan,17450002000185_13673.9_2023-11-16
119,solicitado,Perez & Filho LTDA,17450002000185,"16 Nov, 2023, 15:56",e7084098202311161830vj4shflpoby,"12,343.09",2023-11-16,jhonatan Pereira barcelos,70260107271,CONTA_PAGAMENTO,17339086,1,290.00,8561701,Pagseguro Internet S.A,nan,nan,17450002000185_12343.09_2023-11-16


In [52]:
df[df['transaction_payer_registration_data_name']== 'Royalt Tech LTDA'].sort_values('transaction_amount_total', ascending=False)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl,time_diff,hora,dia,dia_semana,test_transaction,chave,range
2775469,2024-07-24 03:05:52.217800,c5c454e3-11b9-54c4-954b-af31572750d2,transfeera,1ef4969a-28c2-669e-9310-cfab314d6dc4,,BRL,"25,000.00",2024-07-24,cash-out,pix,1ef27f53-2bab-60f9-839e-8d34a78999cb,Royalt Tech LTDA,,51957764000100,04121002,Sao Paulo,Rua Santa Cruz,2187,SALA 10,BR,CHECKING_ACCOUNT,593,[],[],None,...,None,REJECTED,2024-07-24 03:05:52.217943,RULE,[757],None,None,None,1,cnpj,51957764000100,None,None,None,None,None,None,NaN,0 days 00:00:14.316000,3,24,2,0,51957764000100_25000.0_2024-07-24,NaN
2721013,2024-07-23 10:45:51.159464,93e9ce6a-8f64-5a1d-b9e0-73fb61edf64d,transfeera,1ef48e0b-a653-6005-9310-1843bdd3a173,,BRL,"24,400.51",2024-07-23,cash-out,pix,1ef27f53-2bab-60f9-839e-8d34a78999cb,Royalt Tech LTDA,,51957764000100,04121002,Sao Paulo,Rua Santa Cruz,2187,SALA 10,BR,CHECKING_ACCOUNT,593,[],[],None,...,None,REJECTED,2024-07-23 10:45:51.159621,RULE,[757],None,None,None,1,cnpj,51957764000100,None,None,None,None,None,None,NaN,0 days 00:00:16.809000,10,23,1,0,51957764000100_24400.51_2024-07-23,NaN
4630726,2024-08-20 16:06:00.282094,312947ec-f177-5249-b83c-67bf846dc398,transfeera,E27084098202408201605H0oJgQiPmDz,,BRL,"22,994.52",2024-08-20,cash-out,pix,1ef34000-6eb4-6aea-839e-e223da72308c,Royalt Tech LTDA,,51957764000100,04121002,Sao Paulo,Rua Santa Cruz,2187,SALA 10,BR,CHECKING_ACCOUNT,593,[],[],49525496821,...,None,REJECTED,2024-08-20 16:06:00.282386,RULE,[757],None,None,None,1,cnpj,51957764000100,cpf,49525496821,{},None,None,null,"22,994.52",0 days 00:00:03.293000,16,20,1,0,51957764000100_22994.52_2024-08-20,0.00-500000.00
5111516,2024-08-27 17:02:09.338699,2cf7d285-7daf-5eea-af29-d63a1b179813,transfeera,E27084098202408271702Dry1Upqd34j,,BRL,"22,063.00",2024-08-27,cash-out,pix,1ef5a88d-e5d3-6579-8d2d-17d8eea0a52f,Royalt Tech LTDA,,51957764000100,04121002,Sao Paulo,Rua Santa Cruz,2187,SALA 10,BR,CHECKING_ACCOUNT,593,[],[],38637179886,...,None,APPROVED,2024-08-27 17:02:09.338938,RULE,"[757, 956]",None,None,None,1,cnpj,51957764000100,cpf,38637179886,{},None,None,null,"22,063.00",0 days 00:00:07.275000,17,27,1,0,51957764000100_22063.0_2024-08-27,0.00-500000.00
3734812,2024-08-07 00:48:12.861051,86bf8f4d-03c8-5f68-8a8b-820d8531fbd9,transfeera,E27084098202408070048k8EAIUQkATo,,BRL,"21,267.51",2024-08-07,cash-out,pix,1ef46bed-bfba-64f4-9656-cfe3b8779a43,Royalt Tech LTDA,,51957764000100,04121002,Sao Paulo,Rua Santa Cruz,2187,SALA 10,BR,CHECKING_ACCOUNT,593,[],[],52251518000100,...,None,REJECTED,2024-08-07 00:48:12.861280,RULE,[757],None,None,None,1,cnpj,51957764000100,cnpj,52251518000100,None,None,None,null,NaN,0 days 00:00:00.616000,0,7,2,0,51957764000100_21267.51_2024-08-07,NaN
...,...,...,...,...,...,...,...,.

In [53]:
## trx em 2024 
df_fraud[df_fraud['Transação finalizada em'] > '2023-12-31' ].shape

(111, 18)

In [54]:
df_fraud['Transação finalizada em'].describe()

count                              133
mean     2024-04-08 12:27:04.060150272
min                2023-11-10 00:00:00
25%                2024-02-08 00:00:00
50%                2024-03-14 00:00:00
75%                2024-07-03 00:00:00
max                2024-10-25 00:00:00
Name: Transação finalizada em, dtype: object

In [55]:
df_fraud.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 18 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Tipo de MED                                     133 non-null    object        
 1   Nome da conta                                   133 non-null    object        
 2   CNPJ da conta                                   133 non-null    object        
 3   Pedido de infração criado em                    133 non-null    object        
 4   End2end da transação                            133 non-null    object        
 5   Valor da transação                              133 non-null    float64       
 6   Transação finalizada em                         133 non-null    datetime64[ns]
 7   Dados do recebedor - Nome                       133 non-null    object        
 8   Dados do recebedor - Documento                  13

In [56]:
df_fraud['Valor da transação'].describe(percentiles=[.1,.25,.5, .75, .9, .99])

count      133.00
mean    11,348.34
std     12,276.83
min          1.00
10%        154.00
25%      1,004.13
50%      6,864.60
75%     20,000.00
90%     25,000.00
99%     41,574.80
max     70,362.60
Name: Valor da transação, dtype: float64

In [57]:
df_fraud['Transação finalizada em'] = pd.to_datetime(df_fraud['Transação finalizada em'])
df_fraud['Transação finalizada em'].describe()

count                              133
mean     2024-04-08 12:27:04.060150272
min                2023-11-10 00:00:00
25%                2024-02-08 00:00:00
50%                2024-03-14 00:00:00
75%                2024-07-03 00:00:00
max                2024-10-25 00:00:00
Name: Transação finalizada em, dtype: object

In [58]:
df_fraud = df_fraud.sort_values(by=['CNPJ da conta', 
                        'Dados do recebedor - Documento', 
                        'Transação finalizada em'])
df_fraud['payer->receiver'] = df_fraud.groupby(
    ['CNPJ da conta', 'Dados do recebedor - Documento']
)['Transação finalizada em'].diff()
tempo_medio_entre_trx = df_fraud.groupby(
    ['CNPJ da conta', 'Dados do recebedor - Documento']
)['payer->receiver'].mean()
pd.DataFrame(tempo_medio_entre_trx).describe(percentiles=[.1,.25,.5, .75, .9, .99])

,payer->receiver
count,19
mean,0 days 13:09:28.421052631
std,1 days 13:36:50.113903535
min,0 days 00:00:00
10%,0 days 00:00:00
25%,0 days 00:00:00
50%,0 days 00:00:00
75%,0 days 00:00:00
90%,1 days 22:24:00
99%,5 days 16:33:36


In [59]:
## para regras


df_fraud = df_fraud.sort_values(by=['CNPJ da conta', 
                        'Transação finalizada em'])
df_fraud['payer'] = df_fraud.groupby(
    ['CNPJ da conta']
)['Transação finalizada em'].diff()
tempo_medio_entre_trx_all = df_fraud.groupby(
    ['CNPJ da conta']
)['payer'].max()
pd.DataFrame(tempo_medio_entre_trx_all).describe(percentiles=[.1,.25,.5, .75, .9, .99])

,payer
count,15
mean,6 days 03:12:00
std,11 days 13:21:20.436320254
min,0 days 00:00:00
10%,0 days 00:00:00
25%,0 days 00:00:00
50%,0 days 00:00:00
75%,10 days 00:00:00
90%,15 days 04:48:00
99%,38 days 08:38:23.999999999


### Cria Colunas de Agregação e Velocidade

In [60]:
# sample_size = int(df.shape[0]*.1)
# df_test_query = df.sample(sample_size, random_state=seed)

In [61]:
# sql = duckdb.connect()
# df_feat_engineering = sql.query(
# """
# SELECT *,   
#     MEDIAN(transaction_amount_total_brl) OVER (
#         PARTITION BY transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '60 DAYS' PRECEDING AND CURRENT ROW
#     ) AS median_amount_in_60D,    
    
    
#     COUNT(transaction_id) OVER (
#         PARTITION BY transaction_payer_registration_data_doc 
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '1' HOUR  PRECEDING AND CURRENT ROW
#     ) AS count_sent_same_doc_within_1H,   
#     COUNT(transaction_id) OVER (
#         PARTITION BY transaction_payer_registration_data_doc 
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '1' DAY  PRECEDING AND CURRENT ROW
#     ) AS count_sent_same_doc_within_1D,   
#     COUNT(transaction_id) OVER (
#         PARTITION BY transaction_payer_registration_data_doc 
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '7' DAY  PRECEDING AND CURRENT ROW
#     ) AS count_sent_same_doc_within_7D,   
#     COUNT(transaction_id) OVER (
#         PARTITION BY transaction_payer_registration_data_doc 
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '14' DAY  PRECEDING AND CURRENT ROW
#     ) AS count_sent_same_doc_within_14D,  
#     COUNT(transaction_id) OVER (
#         PARTITION BY transaction_payer_registration_data_doc 
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '30' DAY  PRECEDING AND CURRENT ROW
#     ) AS count_sent_same_doc_within_30D,  
    
    
#     SUM(transaction_amount_total_brl) OVER (
#         PARTITION BY  transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '1' HOUR  PRECEDING AND CURRENT ROW
#     ) AS sum_sent_same_doc_within_1H,
#     SUM(transaction_amount_total_brl) OVER (
#         PARTITION BY  transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '1' DAY  PRECEDING AND CURRENT ROW
#     ) AS sum_sent_same_doc_within_1D,
#     SUM(transaction_amount_total_brl) OVER (
#         PARTITION BY  transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '7' DAY  PRECEDING AND CURRENT ROW
#     ) AS sum_sent_same_doc_within_7D,
#     SUM(transaction_amount_total_brl) OVER (
#         PARTITION BY  transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '14' DAY  PRECEDING AND CURRENT ROW
#     ) AS sum_sent_same_doc_within_14D,
#     SUM(transaction_amount_total_brl) OVER (
#         PARTITION BY  transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '30' DAY  PRECEDING AND CURRENT ROW
#     ) AS sum_sent_same_doc_within_30D,


#     SUM(test_transaction) OVER (
#         PARTITION BY transaction_payer_registration_data_doc
#         ORDER BY transaction_datetime_request
#         RANGE BETWEEN INTERVAL '30' DAY PRECEDING AND CURRENT ROW
#     ) AS count_test_in_30D_same_payer_and_receiver,


# FROM df_test_query
# ORDER BY transaction_payer_registration_data_doc, transaction_datetime_request;
# """
# ).df().fillna(0)

In [ ]:
from time import *

toc = time()

def convert_seconds(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

time_format = convert_seconds(toc-tic)

print(f'Tempo de Processamento: {time_format}')